In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns

from scipy import stats
from scipy.stats import  norm
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')


In [2]:
datasets = pd.read_csv("/home/luming/kaggle/house-prices-advanced-regression-techniques/data/train.csv")
testsets = pd.read_csv("/home/luming/kaggle/house-prices-advanced-regression-techniques/data/test.csv")
from sklearn.impute import SimpleImputer
data_index=datasets.iloc[0]

In [3]:
datasets[data_index.index[0]].iloc[:]

0          1
1          2
2          3
3          4
4          5
        ... 
1455    1456
1456    1457
1457    1458
1458    1459
1459    1460
Name: Id, Length: 1460, dtype: int64

In [19]:
for i in range(len(data_index.index)):
    datasets[data_index.index[i]].iloc[:].fillna(value=99999.0,inplace=True)
datasets

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,99999,Reg,Lvl,AllPub,...,0,99999,99999,99999,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,99999,Reg,Lvl,AllPub,...,0,99999,99999,99999,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,99999,IR1,Lvl,AllPub,...,0,99999,99999,99999,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,99999,IR1,Lvl,AllPub,...,0,99999,99999,99999,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,99999,IR1,Lvl,AllPub,...,0,99999,99999,99999,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,99999,Reg,Lvl,AllPub,...,0,99999,99999,99999,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,99999,Reg,Lvl,AllPub,...,0,99999,MnPrv,99999,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,99999,Reg,Lvl,AllPub,...,0,99999,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,99999,Reg,Lvl,AllPub,...,0,99999,99999,99999,0,4,2010,WD,Normal,142125


In [20]:
vars_info=pd.read_csv("/home/luming/kaggle/house-prices-advanced-regression-techniques/data/variable_info.csv", index_col=0)
vars_info


,Id,correlate,continuous,Chinese_notes
1,MSSubClass,0.0,1.0,房子的類型（適合怎樣的家庭組成）
2,MSZoning,0.0,0.0,土地利用情形
3,LotFrontage,0.0,0.0,連接幾吋街道
4,LotArea,0.0,1.0,有幾平方英尺
5,Street,0.0,0.0,鄰近街道的鋪面
...,...,...,...,...
76,MoSold,0.0,1.0,售出月份
77,YrSold,0.0,1.0,售出年份
78,SaleType,0.0,0.0,售出類型
79,SaleCondition,0.0,0.0,售出原因


In [6]:
x_train=datasets.iloc[:, :80]
y_train=datasets.iloc[:,-1]


In [7]:
from itertools import product

In [8]:

groups = {}
i = [0,1]
j = [0,1]
for x, y in product(i, j):
    tmp = vars_info[(vars_info.correlate == x).to_numpy() & (vars_info.continuous == y).to_numpy()]['Id'].to_numpy()
    groups[f'group_{x}_{y}'] = tmp

In [9]:
groups

{'group_0_0': array(['MSZoning', 'LotFrontage', 'Street', 'Alley', 'LotShape',
        'LandContour', 'Utilities', 'LandSlope', 'Neighborhood',
        'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
        'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
        'RoofStyle', 'RoofMatl', 'E0terior1st', 'E0terior2nd',
        'MasVnrType', 'MasVnrArea', 'E0terQual', 'E0terCond', 'Foundation',
        'BsmtQual', 'BsmtCond', 'BsmtE0posure', 'BsmtFinType1',
        'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
        'KitchenQual', 'Functional', 'Fireplaces', 'FireplaceQu',
        'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
        'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType',
        'SaleCondition', 'SalePrice'], dtype=object),
 'group_0_1': array(['MSSubClass', 'LotArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
        'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
        'BsmtFullBath', 'BsmtHa

In [12]:
from sklearn.preprocessing import LabelEncoder

In [21]:
#print(groups['group_0_1'][0])
lb_make = LabelEncoder()
for i in range(len(groups['group_0_1'])):
    x_train[groups['group_0_1'][i]+'_code'] = lb_make.fit_transform(x_train[groups['group_0_1'][i]])
    

In [23]:
from sklearn.preprocessing import OneHotEncoder


In [22]:

for i in range(len(groups['group_0_0'])):
    onehotencoder=OneHotEncoder(x_train[groups['group_0_0'][i]])
    x_train=onehotencoder.fit_transform(x_train)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().